In [1]:
!pip install quacc matcalc

Here, we will demonstrate how to model surfaces. We will study the adsorption of N2 on Fe. For this analysis, we will use the TensorNet-MatPES-r2SCAN machine learning potential we used in a prior exercise. Again, the choice of calculator can be easily swapped and is not the main focal point of this exercise.

The question we want to answer is: how strongly does N2 adsorb on the surface of iron, and what is the most stable adsorption geometry?


In [2]:
potential_name = "TensorNet-MatPES-r2SCAN-v2025.1-PES"

Our first order of business is to construct a surface of Fe. We start by taking the bulk structure of Fe and relaxing it. The bulk structure of Fe could be obtained from an experimental crystal structure, from the Materials Project, or any other place. We will take the one provideed by ASE for convenience.


In [3]:
from ase.build import bulk

crystal = bulk("Fe")

Now we attach the calculator and relax the bulk Fe structure. We make sure to relax both the positions and unit cell here.


In [4]:
from ase.filters import FrechetCellFilter
from ase.optimize import BFGS
from matcalc import load_fp

crystal.calc = load_fp(potential_name)  # Assign a calculator
crystal_wrapped = FrechetCellFilter(crystal)  # Tell ASE to optimize cell too
opt = BFGS(crystal_wrapped, trajectory="bulk_relax.traj")  # Set up optimizer
opt.run(fmax=0.01)  # Run optimization until forces < 0.01 eV/Å

      Step     Time          Energy          fmax
BFGS:    0 21:23:56      -14.402250        0.111271
BFGS:    1 21:23:56      -14.402630        0.047494
BFGS:    2 21:23:56      -14.402715        0.000793


np.True_

Now that we have a relaxed bulk structure, we can carve a surface. There are many tools to carve surfaces in Pymatgen, but I have already coded some convenient utilities in my code quacc. We will use those for simplicity. As always, we should look at the arguments and docstrings to understand what the function does.


In [5]:
from quacc.atoms.slabs import make_slabs_from_bulk

slabs = make_slabs_from_bulk(crystal)

We have generated a bunch of plausible surface slabs. Let's view some of them. Notice how some of the bottom layers are held fixed. If you wanted to do this yourself, you could do so with `ase.constraints.FixAtoms`.


In [6]:
from ase.visualize import view

view(slabs)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Now that we have a bunch of surface slabs, which do we pick?

We generally must find the one with the lowest energy. So, now we must relax each surface _at a fixed cell volume_ and find the one with the lowest energy.

Technically, we should identify the surface with the lowest surface energy rather than total energy, but for simplicity we will ignore that since it will not change our approach.

Remember that we cannot compare energies if the compositions are different, so we will have to normalize by the number of atoms.


In [7]:
for slab in slabs:
    slab.calc = load_fp(potential_name)
    opt = BFGS(slab)
    opt.run(fmax=0.01)

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


      Step     Time          Energy          fmax
BFGS:    0 21:23:58     -728.335999        0.744082
BFGS:    1 21:23:59     -728.385742        0.696431
BFGS:    2 21:23:59     -728.637390        0.185446
BFGS:    3 21:23:59     -728.640808        0.117617
BFGS:    4 21:23:59     -728.642700        0.094741
BFGS:    5 21:24:00     -728.650391        0.053678
BFGS:    6 21:24:00     -728.651489        0.015681
BFGS:    7 21:24:00     -728.651611        0.012194
BFGS:    8 21:24:01     -728.651672        0.010710


INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:    9 21:24:01     -728.651672        0.004956
      Step     Time          Energy          fmax
BFGS:    0 21:24:02    -1008.786133        0.576584
BFGS:    1 21:24:02    -1008.836426        0.532177
BFGS:    2 21:24:02    -1009.072021        0.172038
BFGS:    3 21:24:03    -1009.077759        0.146461
BFGS:    4 21:24:03    -1009.101562        0.116937
BFGS:    5 21:24:03    -1009.113770        0.070169
BFGS:    6 21:24:04    -1009.117798        0.062186
BFGS:    7 21:24:04    -1009.118774        0.058084
BFGS:    8 21:24:05    -1009.120728        0.068616
BFGS:    9 21:24:05    -1009.123779        0.071683
BFGS:   10 21:24:05    -1009.127869        0.052392
BFGS:   11 21:24:06    -1009.130493        0.050126
BFGS:   12 21:24:06    -1009.131226        0.040793
BFGS:   13 21:24:06    -1009.131592        0.034786
BFGS:   14 21:24:07    -1009.132141        0.029125
BFGS:   15 21:24:07    -1009.132935        0.028413
BFGS:   16 21:24:07    -1009.133850        0.020508
BFGS:   17 21:

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:   18 21:24:08    -1009.134399        0.002138
      Step     Time          Energy          fmax
BFGS:    0 21:24:08     -587.966553        0.397252
BFGS:    1 21:24:08     -587.982422        0.335106


INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:    2 21:24:09     -588.021362        0.011880
BFGS:    3 21:24:09     -588.021362        0.004796
      Step     Time          Energy          fmax
BFGS:    0 21:24:09    -1123.056030        0.113668
BFGS:    1 21:24:10    -1123.058838        0.105529
BFGS:    2 21:24:10    -1123.076050        0.004755


In [8]:
for i, slab in enumerate(slabs):
    e = slab.get_potential_energy() / len(slab)
    print(f"Slab {i} energy: {e} eV/atom")

Slab 0 energy: -14.012532160832333 eV/atom
Slab 1 energy: -14.015755547417534 eV/atom
Slab 2 energy: -14.000508626302084 eV/atom
Slab 3 energy: -14.038450622558594 eV/atom


We will continue with the lowest energy surface here.


In [9]:
import numpy as np

min_energy_index = np.argmin([slab.get_potential_energy() for slab in slabs])
slab = slabs[min_energy_index]

Now we need to add an adsorbate to our slab. Let's start by defining the N2 molecule.


In [10]:
from ase.build import molecule
from ase.optimize import BFGS

adsorbate = molecule("N2")
adsorbate.calc = load_fp(potential_name)
opt = BFGS(adsorbate)
opt.run(fmax=0.01)

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


      Step     Time          Energy          fmax
BFGS:    0 21:24:10      -17.851177        3.448449
BFGS:    1 21:24:11      -17.394592       14.897353
BFGS:    2 21:24:11      -17.892418        0.943796
BFGS:    3 21:24:11      -17.895245        0.237782
BFGS:    4 21:24:11      -17.895433        0.005883


np.True_

Now we need to add our adsorbate to our surface. There are many possible surface sites. The only way to know where the adsorbate should go is to add the adsorbate to all plausible surface sites, do a structure relaxation, and identify the lowest energy configuration(s). There are tools in Pymatgen to do this, but again we will use a utility function I have developed in quacc.


In [11]:
from quacc.atoms.slabs import make_adsorbate_structures

slab_adsorbates = make_adsorbate_structures(slab, adsorbate)

We now have many slab-adsorbate systems with the adsorbate at various sites.


In [12]:
view(slab_adsorbates)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Time to relax each one to find the lowest energy configuration for the adsorbate. We do not need to normalize on a per-atom basis here because all the systems have the same number of atoms since we are using the same surface for each calculation.


In [13]:
from ase.optimize import BFGS

for slab_adsorbate in slab_adsorbates:
    slab_adsorbate.calc = load_fp(potential_name)
    opt = BFGS(slab_adsorbate)
    opt.run(fmax=0.01)

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


      Step     Time          Energy          fmax
BFGS:    0 21:24:12    -1141.104980        2.813444
BFGS:    1 21:24:13    -1141.014648        4.891023
BFGS:    2 21:24:13    -1141.159424        0.797703
BFGS:    3 21:24:14    -1141.177124        0.641555
BFGS:    4 21:24:14    -1141.200928        0.922816
BFGS:    5 21:24:15    -1141.218994        0.657942
BFGS:    6 21:24:15    -1141.221069        0.242027
BFGS:    7 21:24:15    -1141.222290        0.263479
BFGS:    8 21:24:16    -1141.223755        0.428557
BFGS:    9 21:24:16    -1141.225952        0.456410
BFGS:   10 21:24:16    -1141.227173        0.269817
BFGS:   11 21:24:17    -1141.227905        0.048570
BFGS:   12 21:24:17    -1141.228394        0.174394
BFGS:   13 21:24:18    -1141.229248        0.288200
BFGS:   14 21:24:18    -1141.229980        0.264584
BFGS:   15 21:24:18    -1141.230469        0.113906
BFGS:   16 21:24:19    -1141.230591        0.032681
BFGS:   17 21:24:19    -1141.230713        0.055946
BFGS:   18 21:

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:   20 21:24:21    -1141.230713        0.004345
      Step     Time          Energy          fmax
BFGS:    0 21:24:22    -1140.879028        3.040851
BFGS:    1 21:24:22    -1140.786499        4.962250
BFGS:    2 21:24:23    -1140.931519        1.050057
BFGS:    3 21:24:24    -1140.947998        0.565484
BFGS:    4 21:24:25    -1140.968140        1.002244
BFGS:    5 21:24:25    -1140.999268        0.863521
BFGS:    6 21:24:26    -1141.003418        0.401521
BFGS:    7 21:24:26    -1141.006104        0.292567
BFGS:    8 21:24:27    -1141.009399        0.582148
BFGS:    9 21:24:28    -1141.014771        0.789636
BFGS:   10 21:24:28    -1141.019775        0.677056
BFGS:   11 21:24:29    -1141.023315        0.330137
BFGS:   12 21:24:29    -1141.025879        0.174100
BFGS:   13 21:24:30    -1141.027832        0.378750
BFGS:   14 21:24:30    -1141.030884        0.481097
BFGS:   15 21:24:31    -1141.033081        0.318842
BFGS:   16 21:24:31    -1141.034058        0.078411
BFGS:   17 21:

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:   26 21:24:35    -1141.035278        0.008923
      Step     Time          Energy          fmax
BFGS:    0 21:24:36    -1140.630859        2.931492
BFGS:    1 21:24:36    -1140.531860        4.932026
BFGS:    2 21:24:37    -1140.679443        0.928404
BFGS:    3 21:24:37    -1140.692871        0.439800
BFGS:    4 21:24:37    -1140.707886        0.780145
BFGS:    5 21:24:38    -1140.735352        0.654863
BFGS:    6 21:24:38    -1140.738892        0.268889
BFGS:    7 21:24:39    -1140.741211        0.248275
BFGS:    8 21:24:39    -1140.744629        0.522101
BFGS:    9 21:24:39    -1140.748901        0.619986
BFGS:   10 21:24:40    -1140.752808        0.451529
BFGS:   11 21:24:40    -1140.755981        0.149325
BFGS:   12 21:24:41    -1140.759033        0.369569
BFGS:   13 21:24:41    -1140.763550        0.644333
BFGS:   14 21:24:42    -1140.769043        0.729455
BFGS:   15 21:24:42    -1140.773804        0.507672
BFGS:   16 21:24:42    -1140.778564        0.130714
BFGS:   17 21:

INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\state.pt...
INFO:c:\Users\asros\miniconda3\envs\cms\Lib\site-packages\matgl\utils\io.py:Using cached local file at C:\Users\asros\.cache\matgl\TensorNet-MatPES-r2SCAN-v2025.1-PES\model.json...


BFGS:   31 21:24:49    -1140.810181        0.006292
      Step     Time          Energy          fmax
BFGS:    0 21:24:50    -1140.757812        3.058002
BFGS:    1 21:24:50    -1140.658936        4.989618
BFGS:    2 21:24:50    -1140.806396        1.007305
BFGS:    3 21:24:51    -1140.819946        0.616288
BFGS:    4 21:24:51    -1140.836670        0.977635
BFGS:    5 21:24:52    -1140.881348        0.962821
BFGS:    6 21:24:52    -1140.892212        0.511483
BFGS:    7 21:24:52    -1140.903198        0.582371
BFGS:    8 21:24:53    -1140.926270        1.708068
BFGS:    9 21:24:53    -1140.953369        2.130585
BFGS:   10 21:24:54    -1141.008789        1.889343
BFGS:   11 21:24:54    -1141.030640        0.965714
BFGS:   12 21:24:54    -1141.055054        0.655430
BFGS:   13 21:24:55    -1141.065796        1.429600
BFGS:   14 21:24:55    -1141.083130        0.700658
BFGS:   15 21:24:56    -1141.099121        0.948826
BFGS:   16 21:24:56    -1141.104858        0.146109
BFGS:   17 21:

In [14]:
for i, slab_adsorbate in enumerate(slab_adsorbates):
    print(f"System {i} energy: {slab_adsorbate.get_potential_energy()} eV")

System 0 energy: -1141.230712890625 eV
System 1 energy: -1141.0352783203125 eV
System 2 energy: -1140.8101806640625 eV
System 3 energy: -1141.230712890625 eV


In [15]:
view(slab_adsorbates)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

We will continue with the lowest energy configuration.


In [16]:
min_idx = np.argmin(
    [slab_adsorbate.get_potential_energy() for slab_adsorbate in slab_adsorbates]
)
final_system = slab_adsorbates[min_idx]

In [17]:
view(final_system)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

We can be thorough and confirm that the structure we have is indeed a local minimum by carrying out a vibrational mode analysis.


In [18]:
from ase.vibrations import Vibrations

vib = Vibrations(final_system)
vib.run()
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    6.0      48.3
  1    6.7      53.9
  2    7.8      63.0
  3   11.3      91.0
  4   11.3      91.1
  5   11.4      92.1
  6   11.5      92.6
  7   11.6      93.3
  8   12.7     102.1
  9   13.4     108.4
 10   14.2     114.9
 11   19.7     159.0
 12   19.7     159.1
 13   19.9     160.2
 14   19.9     160.4
 15   19.9     160.5
 16   19.9     160.6
 17   19.9     160.8
 18   20.0     160.9
 19   20.0     161.6
 20   20.2     162.7
 21   20.4     164.3
 22   20.7     166.8
 23   20.7     166.9
 24   20.9     168.6
 25   21.1     170.4
 26   21.1     170.5
 27   21.3     172.1
 28   21.9     176.5
 29   22.5     181.7
 30   23.0     185.9
 31   23.1     186.3
 32   23.2     186.8
 33   23.2     186.9
 34   23.4     189.0
 35   23.5     189.4
 36   23.6     190.3
 37   24.0     193.3
 38   24.1     194.1
 39   25.0     201.6
 40   25.7     207.2
 41   25.8     208.1
 42   26.3     212.1
 43   29.1     235.1
 44   29.3 

No imaginary modes. Excellent!


Now, we can calculate the adsorption energy.


In [19]:
delta_E = (
    final_system.get_potential_energy()
    - slab.get_potential_energy()
    - adsorbate.get_potential_energy()
)
print(f"Adsorption energy: {delta_E} eV")

Adsorption energy: -0.2592296600341797 eV
